In [ ]:
pip install trimesh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.3/709.3 kB 14.2 MB/s eta 0:00:00


In [ ]:
import os
import glob
import trimesh
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import data as tf_data
from keras import ops
import keras
from keras import layers
from matplotlib import pyplot as plt
from google.colab import drive
drive.mount('/content/drive')
import os
import zipfile
import numpy as np
import pandas as pd
import trimesh
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

Mounted at /content/drive


In [ ]:


# === Décompression du fichier ZIP ===
def unzip_file(zip_path, extract_to):
    """
    Fonction pour décompresser un fichier ZIP dans le répertoire spécifié.

    Arguments:
        zip_path (str): Chemin du fichier ZIP à décompresser.
        extract_to (str): Dossier où les fichiers seront extraits.
    """
    print(f"Décompression de {zip_path} vers {extract_to}...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Décompression terminée. Les fichiers sont extraits dans {extract_to}.")

# ===  Chemin des fichiers et décompression ===
ZIP_FILE_PATH = "/content/drive/MyDrive/shrec2025/off_output.zip"  # Chemin du fichier ZIP
EXTRACT_DIR = "./proteins"  # Dossier d'extraction des fichiers .off

# Décompresser le fichier ZIP
if not os.path.exists(EXTRACT_DIR):
    os.makedirs(EXTRACT_DIR)
unzip_file(ZIP_FILE_PATH, EXTRACT_DIR)


Décompression de /content/drive/MyDrive/shrec2025/off_output.zip vers ./proteins...
Décompression terminée. Les fichiers sont extraits dans ./proteins.


In [ ]:
import os
import pandas as pd
import numpy as np
import trimesh
from tqdm import tqdm
def parse_dataset(label_path, data_path, anonymised=False):
    """
    Fonction pour traiter le jeu de données, lire les fichiers .off et échantillonner des points,
    puis générer les étiquettes correspondantes.

    Arguments:
        label_path (str): Chemin vers le fichier CSV contenant les informations.
        data_path (str): Répertoire contenant les fichiers .off.
        anonymised (bool): Si True, ne lit pas les labels (par défaut False).

    Retourne:
        points (ndarray): Tableau des points (N, ?, 3)
        labels (ndarray or None): Étiquettes associées (N,) ou None si anonymised=True
    """

    df = pd.read_csv(label_path)
    points, labels = [], []

    for _, row in tqdm(df.iterrows(), total=len(df), desc=" Traitement des données"):
        if anonymised:
            anonymised_file_path = os.path.join(data_path, row["anonymised_protein_id"].split('.')[0] + ".off")
            try:
                mesh = trimesh.load(anonymised_file_path)
                points.append(mesh.vertices)
            except Exception as e:
                print(f" Échec du traitement (anonymised): {anonymised_file_path}, Erreur: {e}")
        else:
            file_path = os.path.join(data_path, row["protein_id"] + ".off")
            if os.path.exists(file_path):
                try:
                    mesh = trimesh.load(file_path)
                    points.append(mesh.vertices)
                    labels.append(row["label"])
                except Exception as e:
                    print(f" Échec du traitement: {file_path}, Erreur: {e}")
            else:
                print(f" Fichier manquant: {file_path}")

    points = np.array(points, dtype=np.float32)

    if anonymised:
        return points, None
    else:
        labels = np.array(labels, dtype=np.int32)
        return points, labels


In [ ]:

ZIP_FILE_PATH = "/content/drive/MyDrive/shrec2025/anonymised_test.zip"  # Chemin du fichier ZIP
EXTRACT_DIR = "./anonymised"  # Dossier d'extraction des fichiers .off

# Décompresser le fichier ZIP
if not os.path.exists(EXTRACT_DIR):
    os.makedirs(EXTRACT_DIR)
unzip_file(ZIP_FILE_PATH, EXTRACT_DIR)
anonymised_test_label_path='/content/anonymised/anonymised_test/test_set.csv'
anonymised_test_data_path='/content/anonymised/anonymised_test/data/'
anonymised_test_points, _ = parse_dataset(anonymised_test_label_path, anonymised_test_data_path, anonymised=True)

Décompression de /content/drive/MyDrive/shrec2025/anonymised_test.zip vers ./anonymised...
Décompression terminée. Les fichiers sont extraits dans ./anonymised.


 Traitement des données: 100%|██████████| 2321/2321 [00:19<00:00, 117.32it/s]


In [ ]:
train_label_path='/content/proteins/off_output/train_labels.csv'
train_data_path='/content/proteins/off_output/train/'
train_points, train_labels= parse_dataset(train_label_path,train_data_path)

 Traitement des données: 100%|██████████| 7383/7383 [00:43<00:00, 170.51it/s]


In [ ]:
test_label_path='/content/proteins/off_output/test_labels.csv'
test_data_path='/content/proteins/off_output/test/'
test_points, test_labels= parse_dataset(test_label_path,test_data_path)

 Traitement des données: 100%|██████████| 1861/1861 [00:11<00:00, 166.34it/s]


In [ ]:


print(f"\ Taille du jeu d'entraînement: {train_points.shape}, Taille du jeu de test: {test_points.shape}, Nombre de classes: {len(np.unique(test_labels))}")


\ Taille du jeu d'entraînement: (7383, 2048, 3), Taille du jeu de test: (1861, 2048, 3), Nombre de classes: 97


In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical



BATCH_SIZE = 16





train_dataset = tf.data.Dataset.from_tensor_slices((train_points, train_labels))

train_dataset = train_dataset.shuffle(len(train_points)).batch(BATCH_SIZE)






test_dataset = tf.data.Dataset.from_tensor_slices((test_points, test_labels))

test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Conv1DTranspose, BatchNormalization, GlobalMaxPooling1D, Input
from tensorflow.keras import Model

def pointnet_sa_module(x, mlp_units):
    """
    PointNet++ Set Abstraction Layer (simplified)
    """
    x = Conv1D(mlp_units[0], 1, activation='relu')(x)
    for units in mlp_units[1:]:
        x = Conv1D(units, 1, activation='relu')(x)
    return x

def build_pointnet_convtranspose_autoencoder(input_shape=(2048, 3)):
    inputs = Input(shape=input_shape)

    # Encoder
    x = pointnet_sa_module(inputs, [64, 64, 128])
    x = BatchNormalization()(x)

    x = pointnet_sa_module(x, [128, 128, 256])
    x = BatchNormalization()(x)

    x = pointnet_sa_module(x, [256, 512, 1024])
    global_feat = GlobalMaxPooling1D()(x)  # (batch_size, 1024)

    # Decoder
    x = tf.keras.layers.Reshape((1, 1024))(global_feat)

    # Now upsample with Conv1DTranspose
    x = Conv1DTranspose(512, kernel_size=8, strides=8, padding='same', activation='relu')(x)   # (8, 512)
    x = Conv1DTranspose(256, kernel_size=8, strides=8, padding='same', activation='relu')(x)   # (64, 256)
    x = Conv1DTranspose(128, kernel_size=4, strides=4, padding='same', activation='relu')(x)   # (256, 128)
    x = Conv1DTranspose(64, kernel_size=4, strides=4, padding='same', activation='relu')(x)    # (1024, 64)
    x = Conv1DTranspose(32, kernel_size=2, strides=2, padding='same', activation='relu')(x)    # (2048, 32)

    outputs = Conv1D(3, 1, activation='tanh', padding='same')(x)  # (2048, 3)

    model = Model(inputs, outputs)
    return model

# 构建模型
autoencoder = build_pointnet_convtranspose_autoencoder()
autoencoder.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 2048, 3)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 2048, 64)       │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 2048, 64)       │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 2048, 128)      │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 2048, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 2048, 128)      │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 2048, 128)      │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 2048, 256)      │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 2048, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, 2048, 256)      │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_7 (Conv1D)               │ (None, 2048, 512)      │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_8 (Conv1D)               │ (None, 2048, 1024)     │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 1024)           │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 1, 1024)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_transpose                │ (None, 8, 512)         │     4,194,816 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_transpose_1              │ (None, 64, 256)        │     1,048,832 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_transpose_2              │ (None, 256, 128)       │       131,200 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_transpose_3              │ (None, 1024, 64)       │        32,832 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_transpose_4              │ (None, 2048, 32)       │         4,128 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_9 (Conv1D)               │ (None, 2048, 3)        │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,214,915 (23.71 MB)

 Trainable params: 6,214,147 (23.71 MB)

 Non-trainable params: 768 (3.00 KB)

In [ ]:
import tensorflow as tf

def chamfer_distance(pcd1, pcd2):

    """
    pcd1_expand = tf.expand_dims(pcd1, axis=2)  # (B, N, 1, 3)
    pcd2_expand = tf.expand_dims(pcd2, axis=1)  # (B, 1, M, 3)

    distances = tf.reduce_sum((pcd1_expand - pcd2_expand) ** 2, axis=-1)  # (B, N, M)

    forward_min = tf.reduce_min(distances, axis=2)  # (B, N)
    backward_min = tf.reduce_min(distances, axis=1)  # (B, M)

    forward_mean = tf.reduce_mean(forward_min, axis=1)  # (B,)
    backward_mean = tf.reduce_mean(backward_min, axis=1)  # (B,)
    chamfer_loss = forward_mean + backward_mean  # (B,)

    return tf.reduce_mean(chamfer_loss)  # scalar
import tensorflow as tf

def repulsion_loss(pred, k=5, h=0.03):
    """
    Simple Repulsion Loss to enforce uniformity
    pred: (B, N, 3)
    k: number of nearest neighbors
    h: bandwidth (Gaussian influence)

    Returns: scalar repulsion penalty
    """
    B, N, _ = pred.shape


    point_expand = tf.expand_dims(pred, axis=2)  # (B, N, 1, 3)
    neighbor_expand = tf.expand_dims(pred, axis=1)  # (B, 1, N, 3)
    dists = tf.reduce_sum((point_expand - neighbor_expand) ** 2, axis=-1)  # (B, N, N)


    dists += tf.eye(N)[None, :, :] * 1e6


    _, indices = tf.math.top_k(-dists, k=k)  # indices: (B, N, k)


    idx_expand = tf.expand_dims(indices, axis=-1)  # (B, N, k, 1)
    neighbors = tf.gather(pred, idx_expand, batch_dims=1, axis=1)  # (B, N, k, 3)
    neighbors = tf.squeeze(neighbors, axis=-2)


    center_points = tf.expand_dims(pred, axis=2)  # (B, N, 1, 3)
    diff = center_points - neighbors  # (B, N, k, 3)
    dist_sq = tf.reduce_sum(diff ** 2, axis=-1)  # (B, N, k)

    # 高斯型 penalty
    weight = tf.exp(-dist_sq / (h ** 2))  # (B, N, k)
    repulsion = tf.reduce_mean(weight * dist_sq)

    return repulsion
def chamfer_with_repulsion_loss(y_true, y_pred, repulsion_weight=0.5):
    chamfer = chamfer_distance(y_true, y_pred)
    repulse = repulsion_loss(y_pred)
    return chamfer + repulsion_weight * repulse


In [ ]:
autoencoder.compile(optimizer='adam', loss=chamfer_with_repulsion_loss)


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_point_clouds(original, reconstructed, title=""):
    fig = plt.figure(figsize=(10, 5))

    ax1 = fig.add_subplot(121, projection='3d')
    ax1.scatter(original[:, 0], original[:, 1], original[:, 2], s=1, c='b')
    ax1.set_title("Original")

    ax2 = fig.add_subplot(122, projection='3d')
    ax2.scatter(reconstructed[:, 0], reconstructed[:, 1], reconstructed[:, 2], s=1, c='r')
    ax2.set_title("Reconstructed")

    plt.suptitle(title)
    plt.tight_layout()
    plt.show()


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

class PointCloudVisualizer(tf.keras.callbacks.Callback):
    def __init__(self, data, interval=10, num_samples=3):
        super().__init__()
        self.data = data  # shape: (B, N, 3)
        self.interval = interval
        self.num_samples = num_samples

    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.interval == 0:
            indices = np.random.choice(len(self.data), size=self.num_samples, replace=False)
            for i, idx in enumerate(indices):
                input_pc = self.data[idx]
                input_batch = tf.expand_dims(input_pc, axis=0)  # shape: (1, N, 3)

                reconstructed = self.model.predict(input_batch, verbose=0)[0]  # shape: (N, 3)
                plot_point_clouds(
                    input_pc, reconstructed,
                    title=f"Epoch {epoch} - Sample {idx} (#{i+1})"
                )


In [ ]:

all_points = np.concatenate([train_points, anonymised_test_points], axis=0)  # shape: (N1 + N2, num_points, 3)


indices = np.random.permutation(all_points.shape[0])
shuffled_points = all_points[indices]


In [ ]:

vis_callback = PointCloudVisualizer(test_points, interval=10)
checkpoint_callback = ModelCheckpoint(
    '/content/drive/MyDrive/shrec2025/proteins/autoencoder1.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='auto',
    verbose=1
)
autoencoder.fit(
    shuffled_points, shuffled_points,
    validation_data=(test_points, test_points),
    epochs=50,
    batch_size=16,
    callbacks=[vis_callback,checkpoint_callback]
)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.keras.layers import BatchNormalization, GlobalMaxPooling1D, Input


inputs = autoencoder.input


x = pointnet_sa_module(inputs, [64, 64, 128])
x = BatchNormalization()(x)

x = pointnet_sa_module(x, [128, 128, 256])
x = BatchNormalization()(x)


x = pointnet_sa_module(x, [256, 512, 1024])
x = GlobalMaxPooling1D()(x)

encoder = Model(inputs, x)
encoder.summary()

autoencoder.load_weights('/content/drive/MyDrive/shrec2025/proteins/autoencoder1.keras')

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 2048, 3)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_19 (Conv1D)              │ (None, 2048, 64)       │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_20 (Conv1D)              │ (None, 2048, 64)       │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_21 (Conv1D)              │ (None, 2048, 128)      │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 2048, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_22 (Conv1D)              │ (None, 2048, 128)      │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_23 (Conv1D)              │ (None, 2048, 128)      │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_24 (Conv1D)              │ (None, 2048, 256)      │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 2048, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_25 (Conv1D)              │ (None, 2048, 256)      │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_26 (Conv1D)              │ (None, 2048, 512)      │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_27 (Conv1D)              │ (None, 2048, 1024)     │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_2          │ (None, 1024)           │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 803,008 (3.06 MB)

 Trainable params: 802,240 (3.06 MB)

 Non-trainable params: 768 (3.00 KB)

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 70 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
from tensorflow.keras.layers import Dense, Dropout

def build_classifier_from_encoder(encoder, num_classes=10):

    x = encoder.output


    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)

    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)


    outputs = Dense(num_classes, activation='softmax')(x)


    model = Model(encoder.input, outputs)
    return model



In [ ]:

for i, layer in enumerate(encoder.layers):

    encoder.layers[i].set_weights(autoencoder.layers[i].get_weights())

    encoder.layers[i].trainable = False


model = build_classifier_from_encoder(encoder, num_classes=97)




model.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 2048, 3)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_19 (Conv1D)              │ (None, 2048, 64)       │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_20 (Conv1D)              │ (None, 2048, 64)       │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_21 (Conv1D)              │ (None, 2048, 128)      │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 2048, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_22 (Conv1D)              │ (None, 2048, 128)      │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_23 (Conv1D)              │ (None, 2048, 128)      │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_24 (Conv1D)              │ (None, 2048, 256)      │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 2048, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_25 (Conv1D)              │ (None, 2048, 256)      │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_26 (Conv1D)              │ (None, 2048, 512)      │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_27 (Conv1D)              │ (None, 2048, 1024)     │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_2          │ (None, 1024)           │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 97)             │        24,929 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,484,065 (5.66 MB)

 Trainable params: 681,057 (2.60 MB)

 Non-trainable params: 803,008 (3.06 MB)

In [ ]:
model.load_weights('/content/drive/MyDrive/shrec2025/best_model_mlp3.keras')


In [ ]:

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=10,
    min_lr=1e-6,
    verbose=1
)

checkpoint_callback = ModelCheckpoint(
    '/content/drive/MyDrive/shrec2025/best_model_GCNNxd.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='auto',
    verbose=1
)

model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)



model.fit(
    train_dataset,
    epochs=100,
    validation_data=test_dataset,
    callbacks=[checkpoint_callback,lr_scheduler]
)


In [ ]:

loss, accuracy = model.evaluate(test_dataset, verbose=1)


print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


117/117 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.8456 - sparse_categorical_accuracy: 0.7991
Test Loss: 0.9251
Test Accuracy: 0.7845


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

y_true = []
y_pred = []


for x_batch, y_batch in test_dataset:

    predictions = model.predict(x_batch)

    pred_classes = np.argmax(predictions, axis=1)



    y_true.extend(y_batch)
    y_pred.extend(pred_classes)

y_true = np.array(y_true)
y_pred = np.array(y_pred)


print("Accuracy:", accuracy_score(y_true, y_pred))
print("Precision (weighted):", precision_score(y_true, y_pred, average='weighted'))
print("Recall (weighted):", recall_score(y_true, y_pred, average='weighted'))
print("F1-score (weighted):", f1_score(y_true, y_pred, average='weighted'))

print("\nClassification Report:")
print(classification_report(y_true, y_pred))


In [ ]:
import numpy as np
import pandas as pd
import trimesh
import os

df = pd.read_csv(anonymised_test_label_path)
predict = []


all_meshes = []


for _, row in df.iterrows():
    anonymised_file_path = os.path.join(anonymised_test_data_path, row["anonymised_protein_id"].split('.')[0] + ".off")
    mesh = trimesh.load(anonymised_file_path)
    all_meshes.append(mesh.vertices)


all_vertices = np.array(all_meshes)


predictions = model.predict(all_vertices)


predict = np.argmax(predictions, axis=1)


df['pred_classes'] = predict
df.to_csv('/content/drive/MyDrive/shrec2025/encoder_decoder_mlp_preiction_sansjoint.csv')

73/73 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step


In [ ]:
df

,anonymised_protein_id,pred_classes
0,0.vtk,39
1,1.vtk,56
2,2.vtk,8
3,3.vtk,54
4,4.vtk,43
...,...,...
2316,2316.vtk,9
2317,2317.vtk,8
2318,2318.vtk,8
2319,2319.vtk,66


In [ ]:
df

,anonymised_protein_id,pred_classes
0,0.vtk,39
1,1.vtk,86
2,2.vtk,8
3,3.vtk,54
4,4.vtk,43
...,...,...
2316,2316.vtk,9
2317,2317.vtk,8
2318,2318.vtk,8
2319,2319.vtk,66
